In [10]:
import tensorflow as tf

In [11]:
def convert_to_corners(boxes):
    return tf.concat(
        [boxes[..., :2] - boxes[..., 2:] / 2.0, boxes[..., :2] + boxes[..., 2:] / 2.0],
        axis=-1,
    )

$$IOU(\mathcal{A},\mathcal{B}) = \frac{\left|\mathcal{A} \cap \mathcal{B}\right|}{\left| \mathcal{A} \cup \mathcal{B}\right|}.$$

![IoU is the ratio of the intersection area to the union area of two bounding boxes.](img/iou.svg)

In [27]:
def compute_iou(boxes1, boxes2):

    boxes1_corners = convert_to_corners(boxes1)
    boxes2_corners = convert_to_corners(boxes2)
    lu = tf.maximum(boxes1_corners[:, None, :2], boxes2_corners[:, :2])
    rd = tf.minimum(boxes1_corners[:, None, 2:], boxes2_corners[:, 2:])
    intersection = tf.maximum(0.0, rd - lu)
    intersection_area = intersection[:, :, 0] * intersection[:, :, 1]
    boxes1_area = boxes1[:, 2] * boxes1[:, 3]
    boxes2_area = boxes2[:, 2] * boxes2[:, 3]
    union_area = tf.maximum(
        boxes1_area[:, None] + boxes2_area - intersection_area, 1e-8
    )
    return tf.clip_by_value(intersection_area / union_area, 0.0, 1.0)

In [64]:
a = tf.constant([[50.0,50.0,20.0,20.0]])
b = tf.constant([[51.0,50.0,20.0,20.0]])

In [65]:
print(convert_to_corners(a).numpy())
print(convert_to_corners(b).numpy())

[[40. 40. 60. 60.]]
[[41. 40. 61. 60.]]


In [66]:
compute_iou(a,b).numpy()

array([[0.9047619]], dtype=float32)